In [16]:
#!pip install opencv-python
"""
use this instead pip3 install opencv-contrib-python
"""
!pip install opencv-contrib-python

I have downloaded .names, .cfg and .weights file. 
First, I will build a yolov3 with opencv demo. Just to quil demonstrate how it works.

In [17]:
import cv2 as cv
import numpy as np

In [18]:
confidence_threshold = 0.10

NMS:

Input: A list of Proposal boxes B, corresponding confidence scores S and overlap threshold N.

Output: A list of filtered proposals D.

In [19]:
nms_threshold = 0.40

In [20]:
inp_height = 416
inp_width = 416

In [21]:
classes_file = "coco.names"

In [22]:
classes = None

In [23]:
class_ids = []
confidences = []
boxes = []

The rstrip() method returns a copy of the string with trailing characters removed 
split('\n')Split a string into a list where each word is a list item

In [24]:
with open(classes_file,'rt')as f:
    classes = f.read().rstrip('\n').split('n')

In [25]:
model_config = "yolov3.cfg"

In [26]:
model_weights = "yolov3.weights"

# setting up our neural network

In [27]:
neural_net = cv.dnn.readNetFromDarknet(model_config, model_weights)


Ask network to use specific computation backend where it supports

In [28]:
#neural_net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
neural_net.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)


Ask network to make computations on specific target device.
I will change this later on my cuda gpu and see if performs better but for now I will assume it's for raspberry pi

In [29]:
#neural_net.setPreferableTarget(cv.dnn.DNN_BACKEND_OPENCV)
neural_net.setPreferableTarget(cv.dnn.DNN_BACKEND_OPENCV)

setting up opencv window

In [30]:
win_name = "Yolov3 + OpenCV"

In [31]:
cv.namedWindow(win_name, cv.WINDOW_NORMAL)
cv.resizeWindow(win_name, 800,800)

In [32]:
capture = cv.VideoCapture(0)

In [33]:
def getOutputsNames(neural_net):
    layer_names = neural_net.getLayerNames()
    #print(layer_names)
    #print(layer_names[i[0] - 1] for i in neural_net.getUnconnectedOutLayers)
    return [layer_names[i[0] - 1] for i in neural_net.getUnconnectedOutLayers()] #


In [34]:
def drawPred(class_id, confidence, left, top, right, bottom):
    cv.rectangle(frame, (left,top), (right,bottom), (255, 178, 50))
    label = "%2f"%confidence
    
    #get confidence and label for class name
    if classes:
        assert(class_id < len(classes))
        label = "%s:%s"%(classes[class_id] , label)
        
    cv.putText(frame, label, (left,top), cv.FONT_HERSHEY_PLAIN, 2, (0,255,255), 4)


In [35]:
def postProcess(frame, outputs):
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]
    
    for output in outputs:
        for detection in output:
            scores = detection[5:] #list has centerx,centery,width,height,confi,0,1,...80 etc
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                
                center_x = int(detection[0]*frame_width)
                center_y = int(detection[1]*frame_height)
                
                width = int(detection[2]*frame_width)
                height = int(detection[3]*frame_height)
                
                left = int(center_x - width/2)
                top = int(center_y - height/2)
                
                class_ids.append(class_id)
                confidences.append(float(confidence))
                
                boxes.append([left, top, width, height])
                
            indices = cv.dnn.NMSBoxes(boxes, confidences,
                                      confidence_threshold,
                                      nms_threshold)
            
            for i in indices:
                i = i[0]
                box = boxes[i]
                left = box[0]
                top = box[1]
                width = box[2]
                height = box[3]
                
                #we want to draw prediction
                class_ids_i = class_ids[i]
                confidences_i = confidences[i]
                drawPred(class_ids_i,confidences_i,left, top, left + width, top+ height)

Incase you are curious what cv.waitKey does - https://stackoverflow.com/questions/5217519/what-does-opencvs-cvwaitkey-function-do 

In [36]:
while cv.waitKey(1) <0:
    has_frame,frame = capture.read() #capturing
    """blob is binary large object that
    we want to fee to our neural network.
    We convert pixel values to 0..1 by divinding each pixel value with 255(color,remember?).
    The the image is resized depending on what we want without cropping!
    We don't want to do any mean subtraction so just pass [0,0,0,0] in mean parameter else it's 1 y default"""
    blob_from_image = cv.dnn.blobFromImage(frame, 1/255,
                                     (inp_width,inp_height),
                                     [0,0,0,0], crop = False
                                          )
    neural_net.setInput(blob_from_image)
    #outs = neural_net.forward (getOutputsNames(net))
    #net.getUnconnectedOutLayersNames()
    """
    custom functions
    """
    outputs = neural_net.forward(getOutputsNames(neural_net))
    #post processing
    postProcess(frame, outputs)
    
    #showing image
    cv.imshow(win_name, frame)
    

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\dnn\src\dnn.cpp:1322: error: (-215:Assertion failed) preferableBackend != DNN_BACKEND_CUDA || IS_DNN_CUDA_TARGET(preferableTarget) in function 'cv::dnn::dnn4_v20191202::Net::Impl::setUpNet'
